In [1]:
# Load required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import fiona
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.ops import unary_union
from shapely.errors import TopologicalError
from unidecode import unidecode
import glob
import csv
from datetime import datetime
import dask.dataframe as dd
import dask_geopandas as dg
from dask.distributed import Client
import gc
import re

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_61540\2701944437.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [2]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
outdata_f = os.path.join(indata_f, 'OutputData', 'step1_GQA')
if not os.path.exists(outdata_f):
    # Create the folder if it doesn't exist
    os.makedirs(outdata_f)

# 0 PREPARE A LOG FILE FOR QC
log_file = 'log_GQA_Step1_191124.csv'
log_path = os.path.join(outdata_f, log_file)

# Initialize Dask client
client = Client()

# Define engines
engines = {
    'fiona': {'engine': 'fiona'},
    'pyogrio': {'engine': 'pyogrio'},
    'pyogrio+arrow': {'engine': 'pyogrio', 'use_arrow': True}
          
}


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53439 instead
  warnings.warn(


In [3]:
# 1 READ URBAN CENTRES
# Read shapefile
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3B4Copy.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)
uc['CNTR_CODE'].fillna('AA', inplace=True)

# Select cities for processing in this batch
###uc_sel = uc.query('Batch==1.0 & CNTR_CODE != "SE"')
uc_sel = uc.query('Batch>0.0')
uc_sel = uc_sel.sort_values(by='HDENS_CLST')

# Read table to list the cities to process using urban centre code
cities_ls = uc_sel.HDENS_CLST.tolist()
len(cities_ls)

240

In [4]:
# Final GQAs
QGA_Final_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\step2_GQA_Final'
# Read table with HDENS Urban centres information and Agglomerations link
HDENS_AGGL_tbl = pd.read_csv(r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\UrbanCentres_Agglomerations_csv.csv')
# Join uc code field to this table
HDENS_AGGL_tbl = HDENS_AGGL_tbl.merge(uc[['POPL_2021', 'HDENS_CLST']], on='POPL_2021')

# 1 UA DATA FOLDER
ua_data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'

# 2 READ NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')

# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery', 
                     **engines['pyogrio+arrow'],columns=['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry'])

# 3 TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

In [ ]:
codes_simpl = codes[['HDENS_CLST', 'HDENS_NAME', 'UA2018']]
codes_simpl['UA2018'] = codes_simpl['UA2018'].str.strip()

In [ ]:
codes_simpl

In [ ]:
codes_simpl = codes_simpl.drop_duplicates(ignore_index=True)
codes_simpl

In [ ]:
codes_grouped = codes_simpl.groupby(['HDENS_CLST']).size().reset_index(name='count')
codes_grouped

In [ ]:
uc_multiUA = codes_grouped.query('count>1').merge(codes[['HDENS_CLST', 'HDENS_NAME']], how='left', on='HDENS_CLST').drop_duplicates(ignore_index=True)
uc_multiUA['ncm_layer_path'] = 'Noise_20202025_export.gpkg'
uc_multiUA.loc[1:3, 'ncm_layer_path'] = 'Noise_20202025_export_DE_update_v2.gpkg'
uc_multiUA.loc[7, 'ncm_layer_path'] = 'Noise_20202025_export_DE_update_v2.gpkg'
uc_multiUA['ncm_layer_name'] = 0
uc_multiUA.loc[0, 'ncm_layer_name'] = 'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_NL'
uc_multiUA.loc[1:3, 'ncm_layer_name'] = 'ncm_DE_upd'
uc_multiUA.loc[4:5,'ncm_layer_name'] = 'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_DE'
uc_multiUA.loc[6,'ncm_layer_name'] = 'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_CH'
uc_multiUA.loc[7, 'ncm_layer_name'] = 'ncm_DE_upd'
uc_multiUA.loc[8, "HDENS_CLST"] = "GEOSTAT21_901"
uc_multiUA.loc[8, "HDENS_NAME"] = "Helsinki"
uc_multiUA.loc[8, "ncm_layer_path"] = "Noise_20202025_export.gpkg"
uc_multiUA.loc[8, "ncm_layer_name"] = 'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_FI'
uc_multiUA.loc[9, "HDENS_CLST"] = "GEOSTAT21_650"
uc_multiUA.loc[9, "HDENS_NAME"] = "Bilbao"
uc_multiUA.loc[9, "ncm_layer_path"] = "Noise_20202025_export.gpkg"
uc_multiUA.loc[9, "ncm_layer_name"] = 'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_ES'

uc_multiUA

In [ ]:
cities_ls = uc_multiUA.HDENS_CLST.to_list()

In [ ]:
cities_ls

In [ ]:
cities_ls =  [
 'GEOSTAT21_334']

In [ ]:
uc_city_code = 'GEOSTAT21_334'
codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[1].strip()

In [5]:
cities_ls = ['GEOSTAT21_012',
'GEOSTAT21_021',
'GEOSTAT21_302',
'GEOSTAT21_305',
'GEOSTAT21_328',
'GEOSTAT21_344']


In [14]:
counter= 1
agl_error_ls = []

# Loop through test cities
for uc_city_code in cities_ls:
    print(counter)
    start_time = datetime.now()
    print(str(start_time))

    inGQA = os.path.join(QGA_Final_path, '{}_finalGQA.shp'.format(uc_city_code))
    if not os.path.exists(inGQA):
        try:
            urban_center = uc.query(f'HDENS_CLST=="{uc_city_code}"')
            uc_name = codes.query(f'HDENS_CLST=="{uc_city_code}"').HDENS_NAME.values[0].strip().replace(" ", "").replace("/", "-")
            ctry_code = codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[0].strip()[:2]
            print(f"{ctry_code} - {uc_name}")

            agl_city = gpd.clip(agls, urban_center)

            # Check noise contour maps GeoPackage file
            if ctry_code == 'DE':
                ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export_DE_update.gpkg')
            else:
                ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export.gpkg')
            layerName = f'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_{ctry_code}'                   
            ncm = gpd.read_file(ncm_file_path, layer=layerName, columns=['category', 'geometry'], 
                                engine='pyogrio', use_arrow=True, bbox= tuple(agl_city.total_bounds))
            print ("ncm")
            ncm = gpd.clip(ncm, urban_center)

            # Define the list of noisy classes
            noisy_classes = ['Lden5559', 'Lden6064', 'Lden6569', 'Lden7074', 'LdenGreaterThan75']
            print(noisy_classes)
            # Create a condition based on the category column
            condition = ncm['category'].isin(noisy_classes)  # Replace 'category_column' with the actual column name

            # Specify the condition and create a new category column based on the condition
            ncm['noisy'] = 0
            ncm.loc[condition, 'noisy'] = 1
            ncm = ncm[['noisy', 'geometry']]
            print(ncm)
            ncm_dis_dg = dg.from_geopandas(ncm, npartitions=10)
            ncm_dis = ncm_dis_dg.dissolve(by='noisy').compute().reset_index()
            print ("ncm_dis")

            # Perform spatial overlay (intersection) 
            ncm_agl = gpd.overlay(ncm_dis, agl_city, how='intersection')
            print ("ncm_agl")
            
            # Aggregate the area with lower band values (quieter bands)
            ncm_agl_city = gpd.overlay(ncm_agl, agl_city, how='union')
            print ("union")

            ncm_agl_city['noisy'] = ncm_agl_city.noisy.fillna(0)
            print ("fillna")

            # Select a subset of columns of interest
            ncm_dis = ncm_agl_city[['noisy', 'geometry']]
            print(ncm_dis)
                                
            # 3 READ URBAN ATLAS DATA 
            group_combinations = codes.query(f'HDENS_CLST=="{uc_city_code}"')[['HDENS_CLST', 'UA2018']].drop_duplicates()
            group_combinations = group_combinations.groupby(['HDENS_CLST', 'UA2018']).size().reset_index(name='count')
            ua_ls = group_combinations['UA2018'].tolist()
            print(ua_ls)
            if len(ua_ls)==1:
                ua_path = codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[0].strip()
                file_path = os.path.join(ua_data_f, f'{ua_path}\Data\{ua_path}.gpkg')
                # Read the GeoPackage file
                ua = gpd.read_file(file_path, layer= ua_path[:-5], 
                            columns= ['code_2018','Pop2018','geometry'], engine='pyogrio', 
                            use_arrow=True, bbox= tuple(urban_center.total_bounds))
                print ("loaded ua in urban city") 
            else:
                ua_path1 = codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[0].strip()
                ua_path2 = codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[1].strip()
                file_path1 = os.path.join(ua_data_f, f'{ua_path1}\Data\{ua_path1}.gpkg')
                file_path2 = os.path.join(ua_data_f, f'{ua_path2}\Data\{ua_path2}.gpkg')
                # Read the GeoPackage file
                ua1 = gpd.read_file(file_path1, layer= ua_path1[:-5], 
                            columns= ['code_2018','Pop2018','geometry'], 
                            engine='pyogrio', 
                            use_arrow=True, bbox= tuple(urban_center.total_bounds))
                ua2 = gpd.read_file(file_path2, layer= ua_path2[:-5], 
                            columns= ['code_2018','Pop2018','geometry'], 
                            engine='pyogrio', 
                            use_arrow=True, bbox= tuple(urban_center.total_bounds))
                
                ua1 = gpd.clip(ua1, urban_center)
                ua2 = gpd.clip(ua2, urban_center)
                ua = pd.concat([ua1, ua2], ignore_index=True)
                print ("loaded ua in urban city")   
            
            # Select 'green' classes
            uagreen = ua.query('code_2018 == "14100" or code_2018 == "31000"')
            print("Selected 'green' classes")

            # 4 SELECT UA INTERSECTING UC
            # Perform spatial overlay (intersection)
            uagreen_urbc = gpd.overlay(uagreen, urban_center, how='intersection')
            print("Selected UA INTERSECTING UC")

            # 5 IDENTIFY GREEN AREAS EXCLUDED (NOT COVERED BY NCM)
            # Perform spatial overlay (intersection)
            nqgreen = gpd.overlay(uagreen_urbc, ncm_dis, how='intersection') #noisy/quiet green
            print('nqgreen')

            # 6 IDENTIFY QUIET/NOISY AREAS
            ## for statistics need to calculate area again
            # Calculate the area for each geometry and create a new column 'area'
            nqgreen['area_m2'] = nqgreen['geometry'].area
            nqgreen['area_ha'] = round(nqgreen['area_m2']* 0.0001,2)
            nqgreen['area_km2'] = round(nqgreen['area_ha']* 0.01,2)
            nqgreen_area = nqgreen.groupby(['code_2018', 'noisy'])['area_m2'].sum().reset_index()
            nqgreen_area['area_ha'] = round(nqgreen_area['area_m2']* 0.0001,2)
            nqgreen_area['area_km2'] = round(nqgreen_area['area_ha']* 0.01,2)

            # 7 EXPORT GREEN QUIET AREAS (GQA)
            # discriminate noisy and quiet
            nqgreen = nqgreen[['CNTR_CODE', 'HDENS_2011', 'code_2018', 'Pop2018','noisy', 'area_m2', 'area_ha', 'area_km2', 'geometry']]            
            print('subset columns in nqgreen')
            GQA = nqgreen.query('noisy == 0')
            GNA = nqgreen.query('noisy == 1')

            # multipart to single part
            GQA_sp = GQA.explode(index_parts=True)
            print('multipart to single part')
            # Reset index to keep it clean
            GQA_sp = GQA_sp.reset_index(drop=True)

            # Select those areas bigger than 100 m2
            GQA = GQA_sp[GQA_sp.geometry.area > 100]
            print('Select those areas bigger than 100 m2')

            # Export to shapefile
            print ('Export to shapefile')
            GQA.to_file(inGQA, driver='ESRI Shapefile')
            print ("GQA")
    
            # Calculate the duration
            end_time = datetime.now()
            processing_time = end_time - start_time
            print ("str(processing_time)")
            
        except:
            print("Error " + uc_city_code)
            agl_error_ls.append(uc_city_code +" error")
    counter= counter+1

print(agl_error_ls)

1
2024-11-28 15:38:28.688347
EE - Tallinn
ncm
['Lden5559', 'Lden6064', 'Lden6569', 'Lden7074', 'LdenGreaterThan75']
   noisy                                           geometry
5      1  MULTIPOLYGON (((5149934.106 4116475.158, 51499...
3      0  MULTIPOLYGON (((5148038.503 4115152.897, 51480...
1      0  MULTIPOLYGON (((5148300.759 4115052.180, 51482...
4      1  MULTIPOLYGON (((5149879.409 4116249.319, 51498...
6      1  MULTIPOLYGON (((5147298.619 4120212.633, 51472...
0      1  MULTIPOLYGON (((5147954.015 4118035.760, 51479...
2      1  MULTIPOLYGON (((5151702.964 4117820.510, 51516...
ncm_dis
ncm_agl
union
fillna
   noisy                                           geometry
0    1.0  MULTIPOLYGON (((5148465.838 4116665.027, 51484...
1    0.0  MULTIPOLYGON (((5160769.180 4126016.723, 51607...
2    0.0  MULTIPOLYGON (((5161000.000 4127580.340, 51609...
['EE001L1_TALLINN_UA2018_v013']
loaded ua in urban city
Selected 'green' classes
Selected UA INTERSECTING UC
nqgreen
subset columns in 

In [11]:
nqgreen = nqgreen[['CNTR_CODE', 'HDENS_2011', 'code_2018', 'Pop2018','noisy',  'area_m2', 'area_ha', 'area_km2', 'geometry']]            

In [12]:
print('subset columns in nqgreen')
GQA = nqgreen.query('noisy == 0')
GNA = nqgreen.query('noisy == 1')

# multipart to single part
GQA_sp = GQA.explode(index_parts=True)
print('multipart to single part')
# Reset index to keep it clean
GQA_sp = GQA_sp.reset_index(drop=True)

# Select those areas bigger than 100 m2
GQA = GQA_sp[GQA_sp.geometry.area > 100]
print('Select those areas bigger than 100 m2')

# Export to shapefile
print ('Export to shapefile')
GQA.to_file(inGQA, driver='ESRI Shapefile')
print ("GQA")

# Calculate the duration
end_time = datetime.now()
processing_time = end_time - start_time
print ("str(processing_time)")

subset columns in nqgreen
multipart to single part
Select those areas bigger than 100 m2
Export to shapefile
GQA
str(processing_time)


In [8]:
nqgreen

,code_2018,Pop2018,HDENS_CLST,HDENS_NAME,HDENS_2011,POPL_2021,CNTR_CODE,MBRS_CODE_,SHAPE_AREA,SHAPE_LEN,Batch,Area_ha,noisy,geometry,area_m2,area_ha,area_km2
0,31000,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,1.0,"POLYGON ((3947008.569 3096084.889, 3947011.279...",48630.107025,4.86,0.05
1,31000,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,1.0,"MULTIPOLYGON (((3948894.050 3096001.808, 39488...",8870.656998,0.89,0.01
2,14100,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,1.0,"POLYGON ((3947216.059 3096539.971, 3947219.431...",8613.858101,0.86,0.01
3,31000,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,1.0,"MULTIPOLYGON (((3950561.723 3097024.869, 39505...",5633.486038,0.56,0.01
4,14100,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,1.0,"POLYGON ((3949152.471 3097702.781, 3949147.171...",14626.281731,1.46,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,14100,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,0.0,"POLYGON ((3947898.962 3096000.000, 3947899.068...",0.701872,0.00,0.00
83,14100,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,0.0,"POLYGON ((3949202.828 3097000.000, 3949144.131...",1017.711777,0.10,0.00
84,14100,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,0.0,"POLYGON ((3949322.423 3097435.450, 3949179.547...",100126.095105,10.01,0.10
85,14100,0,GEOSTAT21_344,Leuven,GEOSTAT11_344,77260.0,BE,1,21000000.0,24000.0,1.0,2100.0,0.0,"MULTIPOLYGON (((3949441.206 3097120.546, 39494...",12582.135974,1.26,0.01


In [ ]:
urban_center

In [ ]:
GQA

In [ ]:
nqgreen

In [ ]:
uc_multiUA.query(f'HDENS_CLST=="{uc_city_code}"')

In [ ]:
uc_multiUA.query(f'HDENS_CLST=="{uc_city_code}"').ncm_layer_name.values[0]

In [ ]:
layerName

In [ ]:
cities_ls3 = ['GEOSTAT21_650',
 'GEOSTAT21_901']

In [ ]:
cities_ls4 = ['GEOSTAT21_334']

In [ ]:
counter= 1
agl_error_ls = []

# Loop through test cities
for uc_city_code in cities_ls4:
    print(counter)
    start_time = datetime.now()
    print(str(start_time))

    urban_center = uc.query(f'HDENS_CLST=="{uc_city_code}"')
    uc_name = codes_simpl.query(f'HDENS_CLST=="{uc_city_code}"').HDENS_NAME.values[0].strip().replace(" ", "").replace("/", "-")
    ctry_code = codes_simpl.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[0].strip()[:2]
    print(f"{ctry_code} - {uc_city_code} - {uc_name}")

    ua_path1 = codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[0].strip()
    ua_path2 = codes.query(f'HDENS_CLST=="{uc_city_code}"').UA2018.values[1].strip()
    if ua_path1 == 'not available' or ua_path2 == 'not available':
        agl_error_ls.append(uc_city_code +" UA not available")
    
    else:
        inGQA = os.path.join(QGA_Final_path, '{}_finalGQA.shp'.format(uc_city_code))
        if not os.path.exists(inGQA):
            try:


                agl_city = gpd.clip(agls, urban_center)

                # Check noise contour maps GeoPackage file
                ncm_file_path_gpkg = uc_multiUA.query(f'HDENS_CLST=="{uc_city_code}"').ncm_layer_path.values[0]
                ncm_file_path_layer = uc_multiUA.query(f'HDENS_CLST=="{uc_city_code}"').ncm_layer_name.values[0]
                ncm_file_path = os.path.join(indata_f, 'NoiseData', ncm_file_path_gpkg)
                layerName = ncm_file_path_layer           
                ncm = gpd.read_file(ncm_file_path, layer=layerName, columns=['category', 'geometry'], 
                                    engine='pyogrio', use_arrow=True, bbox= tuple(agl_city.total_bounds))
                print ("ncm")
                ncm = gpd.clip(ncm, urban_center)

                # Define the list of noisy classes
                noisy_classes = ['Lden5559', 'Lden6064', 'Lden6569', 'Lden7074', 'LdenGreaterThan75']
                print(noisy_classes)
                # Create a condition based on the category column
                condition = ncm['category'].isin(noisy_classes)  # Replace 'category_column' with the actual column name

                # Specify the condition and create a new category column based on the condition
                ncm['noisy'] = 0
                ncm.loc[condition, 'noisy'] = 1
                ncm = ncm[['noisy', 'geometry']]
                print(ncm)
                ncm_dis_dg = dg.from_geopandas(ncm, npartitions=10)
                ncm_dis = ncm_dis_dg.dissolve(by='noisy').compute().reset_index()
                print ("ncm_dis")

                # Perform spatial overlay (intersection) 
                ncm_agl = gpd.overlay(ncm_dis, agl_city, how='intersection')
                print ("ncm_agl")
                
                # Aggregate the area with lower band values (quieter bands)
                ncm_agl_city = gpd.overlay(ncm_agl, agl_city, how='union')
                print ("union")

                ncm_agl_city['noisy'] = ncm_agl_city.noisy.fillna(0)
                print ("fillna")

                # Select a subset of columns of interest
                ncm_dis = ncm_agl_city[['noisy', 'geometry']]
                print(ncm_dis)
                                    
                # 3 READ URBAN ATLAS DATA       
                file_path1 = os.path.join(ua_data_f, f'{ua_path1}\Data\{ua_path1}.gpkg')
                file_path2 = os.path.join(ua_data_f, f'{ua_path2}\Data\{ua_path2}.gpkg')
                # Read the GeoPackage file
                ua1 = gpd.read_file(file_path1, layer= ua_path1[:-5], 
                            columns= ['country', 'fua_name', 'fua_code','code_2018', 'class_2018', 'geometry'], 
                            engine='pyogrio', 
                            use_arrow=True, bbox= tuple(urban_center.total_bounds))
                ua2 = gpd.read_file(file_path2, layer= ua_path2[:-5], 
                            columns= ['country', 'fua_name', 'fua_code','code_2018', 'class_2018', 'geometry'], 
                            engine='pyogrio', 
                            use_arrow=True, bbox= tuple(urban_center.total_bounds))
                print ("loaded ua in urban city")
                ua1 = gpd.clip(ua1, urban_center)
                ua2 = gpd.clip(ua2, urban_center)
                ua = pd.concat([ua1, ua2], ignore_index=True)                   

                # Select 'green' classes
                uagreen = ua.query('code_2018 == "14100" or code_2018 == "31000"')
                
                # 4 SELECT UA INTERSECTING UC
                # Perform spatial overlay (intersection)
                uagreen_urbc = gpd.overlay(uagreen, urban_center, how='intersection')

                # 5 IDENTIFY GREEN AREAS EXCLUDED (NOT COVERED BY NCM)
                # Perform spatial overlay (intersection)
                nqgreen = gpd.overlay(uagreen_urbc, ncm_dis, how='intersection') #noisy/quiet green
                #not_covered = uagreen_urbc.geometry.difference(uagreen_urbc.geometry.intersection(nqgreen.geometry.unary_union))
                # Filter out empty polygons(not empty polygons)
                #green_not_covered_by_ncm = not_covered[~not_covered.is_empty]

                # save to shapefile
                file_path = os.path.join(outdata_f, f'{uc_city_code}_green_not_covered_by_ncm.shp')
                #green_not_covered_by_ncm.to_file(file_path, driver='ESRI Shapefile')
                print ("green_not_covered_by_ncm")

                # 6 IDENTIFY QUIET/NOISY AREAS
                ## for statistics need to calculate area again
                # Calculate the area for each geometry and create a new column 'area'
                nqgreen['area_m2'] = nqgreen['geometry'].area
                nqgreen['area_ha'] = round(nqgreen['area_m2']* 0.0001,2)
                nqgreen['area_km2'] = round(nqgreen['area_ha']* 0.01,2)
                nqgreen_area = nqgreen.groupby(['code_2018', 'noisy'])['area_m2'].sum().reset_index()
                nqgreen_area['area_ha'] = round(nqgreen_area['area_m2']* 0.0001,2)
                nqgreen_area['area_km2'] = round(nqgreen_area['area_ha']* 0.01,2)

                # 7 EXPORT GREEN QUIET AREAS (GQA)
                # discriminate noisy and quiet
                nqgreen = nqgreen[['country', 'fua_name', 'fua_code', 'HDENS_2011', 'code_2018', 'class_2018', 'noisy',  'area_m2', 'area_ha', 'area_km2', 'geometry']]
                GQA = nqgreen.query('noisy == 0')
                GNA = nqgreen.query('noisy == 1')

                # multipart to single part
                GQA_sp = GQA.explode(index_parts=True)
                # Reset index to keep it clean
                GQA_sp = GQA_sp.reset_index(drop=True)

                # Select those areas bigger than 100 m2
                GQA = GQA_sp[GQA_sp.geometry.area > 100]

                # Export to shapefile
                print ('Export to shapefile')
                GQA.to_file(inGQA, driver='ESRI Shapefile')
                print ("GQA")

                # 8 CREATE CENTROIDS FOR GQA POLYGONS
                # Create a new GeoDataFrame with centroids as points
                GQA_pts = gpd.GeoDataFrame(geometry=GQA['geometry'].centroid)
                GQA_pts['oid'] = GQA.index
                GQA_pts['fua_name'] = GQA.fua_name
                GQA_pts['fua_code'] = GQA.fua_code
                GQA_pts['HDENS_2011'] = GQA.HDENS_2011

                # Export to shapefile
                file_path = os.path.join(outdata_f, f'{uc_city_code}_GQA_centroids.shp')
                GQA_pts.to_file(file_path, driver='ESRI Shapefile')

                print ("GQA_pts")
        
                # Calculate the duration
                end_time = datetime.now()
                processing_time = end_time - start_time

                print ("str(processing_time)")
                
                ## write output values into log file
                uc_km2 = round(uc_city.area.sum()/1000000,2)
                agl_city_km2 = round(agl_city.area.sum()/1000000,2)
                ncm_agl_city_km2 = round(ncm_agl_city.area.sum()/1000000,2)
                ua_km2 = round(ua.area.sum()/1000000,2)
                uagreen_km2 = round(uagreen.area.sum()/1000000,2)
                uagreen_urbc_km2 = round(uagreen_urbc.area.sum()/1000000,2)
                nqgreen_m2 = round(nqgreen.area.sum(),2)
                green_not_covered_by_ncm_m2 = round(green_not_covered_by_ncm.area.sum(),2)
                GQA_m2 = round(GQA.area.sum(),2)
                GNA_m2 = round(GNA.area.sum(),2)
                processing_duration = str(processing_time)

                log_entry = create_log_entry(uc_name, uc_city_code, uc_km2, agl_city_km2, 
                                        ncm_agl_city_km2,ua_km2, uagreen_km2, uagreen_urbc_km2, nqgreen_m2, 
                                        green_not_covered_by_ncm_m2, GQA_m2, GNA_m2, processing_time)
                write_log(log_path, log_entry)

                # Clean up intermediate variables to free memory
                del agl_city, ncm, ncm_agl, ncm_agl_city, ncm_dis, ua, uagreen, uagreen_urbc, nqgreen, green_not_covered_by_ncm, GQA, GNA, GQA_pts
            except:
                print("Error " + uc_city_code)
                agl_error_ls.append(uc_city_code +" Topological error")
    counter= counter+1

print(agl_error_ls)

In [ ]:
#GQA = GQA_sp[GQA_sp.geometry.area > 100]
GQA

In [ ]:
ua

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt


# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))
urban_center.plot(ax=ax, edgecolor='black', facecolor=None, alpha=0.5, label='Layer 1')  # Customize color and alpha
ua.plot(ax=ax, edgecolor='red', facecolor=None, alpha=0.5, label='Layer 2')  # Customize color and alpha
ncm.plot(ax=ax, edgecolor='blue', facecolor=None, alpha=0.5, label='Layer 3') 

# Add a legend and title
plt.legend()
plt.title("Overlay of Two GeoDataFrames")

# Show the plot
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt


# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))
urban_center.plot(ax=ax, edgecolor='black', facecolor='none', alpha=1, label='Layer 1')  # Customize color and alpha
uagreen_urbc.plot(ax=ax, edgecolor='green', facecolor='green', alpha=0.5, label='Layer 2')  # Customize color and alpha


# Add a legend and title
plt.legend()
plt.title("Overlay of Two GeoDataFrames")

# Show the plot
plt.show()

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(6, 6))
urban_center.plot(ax=ax, edgecolor='black', facecolor='none', alpha=1, label='Layer 1')  # Customize color and alpha
nqgreen.plot(ax=ax, edgecolor='green', facecolor='green', alpha=0.5, label='Layer 2')  # Customize color and alpha


# Add a legend and title
plt.legend()
plt.title("Overlay of Two GeoDataFrames")

# Show the plot
plt.show()

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(6, 6))
urban_center.plot(ax=ax, edgecolor='black', facecolor='none', alpha=1, label='Layer 1')  # Customize color and alpha
ncm_dis.plot(ax=ax, column='noisy', legend=True, cmap='brg') 
nqgreen.plot(ax=ax, edgecolor='green', facecolor='green', alpha=0.5, label='Layer 2')  # Customize color and alpha

# Add a legend and title
plt.legend()
plt.title("Overlay of Two GeoDataFrames")

# Show the plot
plt.show()

In [ ]:
ncm_dis

In [ ]:
nqgreen = gpd.overlay(uagreen_urbc, ncm_dis, how='intersection') #noisy/quiet green
not_covered = uagreen_urbc.geometry.difference(uagreen_urbc.geometry.intersection(nqgreen.geometry.unary_union))
# Filter out empty polygons(not empty polygons)
green_not_covered_by_ncm = not_covered[~not_covered.is_empty]

In [ ]:
not_covered = uagreen_urbc.geometry.difference(uagreen_urbc.geometry.intersection(nqgreen.geometry.unary_union))


In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(6, 6))
urban_center.plot(ax=ax, edgecolor='black', facecolor='none', alpha=1, label='Layer 1')  # Customize color and alpha
nqgreen.plot(ax=ax, edgecolor='green', facecolor='green', alpha=0.5, label='Layer 2')  # Customize color and alpha
not_covered.plot(ax=ax, edgecolor='red', facecolor='red', alpha=0.5, label='Layer 3') 

# Add a legend and title
plt.legend()
plt.title("Overlay of Two GeoDataFrames")

# Show the plot
plt.show()

In [ ]:
GQA.plot()

In [ ]:
ncm.plot()